In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import csv
from IPython.display import clear_output

In [6]:
data_path = './data'

In [7]:
train = pd.read_csv(data_path + '/train.csv')
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
test = pd.read_csv(data_path + './test.csv')

In [43]:
y_train = train['label'].values
y_train = keras.utils.to_categorical(y_train, 10)
X_train = train.drop(['label'], axis=1)

In [44]:
X_train = X_train / 255
X_train = X_train.values.reshape(-1, 28, 28, 1)

test = test / 255
test = test.values.reshape(-1, 28, 28, 1)

In [38]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [46]:


datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1)

datagen.fit(X_train)

In [47]:

cnn_model = Sequential([
    Conv2D(64, 3, padding='same', activation='relu', input_shape=X_train[0].shape),
    BatchNormalization(),
    MaxPool2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPool2D(),
    Flatten(),
    Dense(512, 'relu'),
    Dense(256, 'relu'),
    Dense(10, 'softmax')
])

cnn_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_31 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 14, 14, 32)        18464     
_________________________________________________________________
batch_normalization_32 (Batc (None, 14, 14, 32)        128       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 1568)             

In [9]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True)

cnn_history = cnn_model.fit(X_train, y_train, epochs=100,
                            validation_data=(X_valid, y_valid),
                            callbacks=[early_stopping],
                            verbose=2)

clear_output()

NameError: name 'cnn_model' is not defined

In [55]:
submission_predictions = cnn_model.predict(test)

In [56]:
submission_predictions = np.argmax(submission_predictions , axis = 1)

In [57]:
print(submission_predictions)

[2 0 9 ... 3 9 2]


In [58]:
with open('./submission.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    spamwriter.writerow(['ImageId', 'Label'])
    for i, item in enumerate(submission_predictions):
      spamwriter.writerow([i+1, item])